# Explosions and extinctions

Major changes in a domain...

404s are not actually useful as they only show what was looked for. Try a year by year comparison?

<p class="alert alert-warning">Work in progress – this notebook isn't finished yet. Check back later for more...<p>

In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm
import pandas as pd
import requests_cache
import time
import altair as alt

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))
s.mount('http://', HTTPAdapter(max_retries=retries))

In [2]:

def get_total_pages(params):
    these_params = params.copy()
    these_params['showNumPages'] = 'true'
    response = s.get('http://web.archive.org/cdx/search/cdx', params=these_params, headers={'User-Agent': ''})
    return int(response.text)

def query_cdx(url, **kwargs):
    results = []
    page = 0
    params = kwargs
    params['url'] = url
    params['output'] = 'json'
    total_pages = get_total_pages(params)
    print(total_pages)
    with tqdm(total=total_pages-page) as pbar:
        while page < total_pages:
            params['page'] = page
            response = requests.get('http://web.archive.org/cdx/search/cdx', params=params, headers={'User-Agent': ''})
            print(response.url)
            response.raise_for_status()
            if page == 0:
                results = response.json()
            else:
                results += response.json()[1:]
            page += 1
            pbar.update(1)
            time.sleep(0.2)
    return results

In [3]:
results = query_cdx('*.naa.gov.au', )

299


http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=0
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=1
http://web.archive.org/cdx/search/cdx?url=%2A.naa.gov.au&output=json&page=2



KeyboardInterrupt: 

In [ ]:
params = {
    'url': 'naa.gov.au',
    'output': 'json'
}
get_total_pages(params)

In [ ]:
results = query_cdx('naa.gov.au/*')

In [ ]:
df = pd.DataFrame(results[1:], columns=results[0])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Convert the timestamp string into a datetime object
df['date'] = pd.to_datetime(df['timestamp'], errors='coerce')
df.sort_values(by='date', inplace=True, ignore_index=True)

#df.drop_duplicates(subset=['urlkey'], keep='first', inplace=True)

In [ ]:
# Convert the length from a string into an integer
df['length'] = df['length'].astype('int')

In [ ]:
df.shape

## Number of captures per year

In [ ]:
df['year'] = df['date'].dt.year

# This gives an error without fillna
df['year'] = df['year'].fillna(0).astype('int')

# The error is because one of the timestamps is incorrectly formatted -- no day!
df.loc[df['year'] == 0]

We could just feed to whole dataframe to Altair for visualisation, but that's not very efficient. Let's transform it with Pandas first.

In [ ]:
# Group by year and status code
df_year_code = df.loc[df['year'] > 0].groupby('year')['statuscode'].value_counts().to_frame().rename(columns={'statuscode': 'total'}).reset_index()
df_year_code.head()

In [ ]:
alt.Chart(df_year_code).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color='statuscode:N'
).properties(width=700)

In [ ]:
# Group by year and mimetype
df_year_type = df.loc[df['year'] > 0].groupby('year')['mimetype'].value_counts().to_frame().rename(columns={'mimetype': 'total'}).reset_index()
df_year_type.head()

In [ ]:
# It would be good to limit to top mimetypes, grouping all the low ranked values together in Altair
# Something like this? https://altair-viz.github.io/gallery/top_k_with_others.html
# Can't get it to work...

alt.Chart(df_year_type).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color='mimetype:N'
).properties(width=700)

In [ ]:
# Only show top-ten mimetypes

alt.Chart(df_year_type.loc[df_year_type['mimetype'].isin(df['mimetype'].value_counts()[:10].index.to_list())]).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color=alt.Color('mimetype:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

In [ ]:
# Normalise mimetypes by grouping everything outside the top ten into 'Other'
top_types = df['mimetype'].value_counts()[:9].index.to_list()
df_year_type['top_types'] = df_year_type['mimetype'].apply(lambda x: x if x in top_types else 'Other')

In [ ]:
alt.Chart(df_year_type).mark_bar().encode(
    x = 'year:N',
    y = 'total:Q',
    color=alt.Color('top_types:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

## Mimetypes and extensions

In [ ]:
df['mimetype'].value_counts().index.to_list()

In [ ]:
df['extension'] = df['original'].str.extract(r'\.(\w+)$')

In [ ]:
df.loc[df['mimetype'] == 'application/pdf']['extension'].value_counts()[:25]

In [ ]:
df.loc[df['mimetype'] == 'text/html']['extension'].value_counts()[:25]

In [ ]:
df['mimetype'].value_counts()[:25]

## Change by year

How many pages were added or removed each year.

In [ ]:
# Get unique urls for each year
df_years = df.loc[df['statuscode'].isin(['200', '-', '301', '302'])].drop_duplicates(subset=['year', 'urlkey'])

In [ ]:
year_totals = df_years['year'].value_counts().to_frame().reset_index()
year_totals.columns = ['year', 'total']
year_totals.head()

In [ ]:
alt.Chart(year_totals).mark_bar().encode(
    x='year:N',
    y='total'
).properties(width=700)

In [ ]:
df_years['top_types'] = df_years['mimetype'].apply(lambda x: x if x in top_types else 'Other')
year_type_totals = df_years.loc[df_years['year'] > 0].groupby('year')['top_types'].value_counts().to_frame().rename(columns={'top_types': 'total'}).reset_index()
alt.Chart(year_type_totals).mark_bar().encode(
    x='year:N',
    y='total:Q',
    color=alt.Color('top_types:N', scale=alt.Scale(scheme='category20b'))
).properties(width=700)

In [ ]:
df_years.loc[df['mimetype'].isin(['text/html', 'warc/revisit'])].groupby('urlkey')['year'].count().mean()

In [ ]:
import numpy as np
df_year_range = df_years.groupby(['urlkey', 'top_types'])['year'].agg([np.max, np.min])
df_year_range.head()

In [ ]:
df_year_range['range'] = df_year_range['amax'] - df_year_range['amin']
df_year_range.reset_index(inplace=True)

In [ ]:
df_year_range['range'].mean()

In [ ]:
df_year_range.groupby('top_types')['range'].mean()

In [ ]:
alt.Chart(df_year_range).mark_bar().encode(
    y='top_types:N',
    x='mean(range):Q'
)

In [ ]:
years = []
for year in range(1998, 2021):
    if year == 1998:
        added = df_years.loc[df_years['year'] == year].shape[0]
        removed = 0
    else:
        this_year = df_years.loc[df_years['year'] == year]['urlkey']
        last_year = df_years.loc[df_years['year'] == year - 1]['urlkey']
        added = this_year.loc[~this_year.isin(last_year)].shape[0]
        removed = last_year.loc[~last_year.isin(this_year)].shape[0]
    years.extend([{'year': year, 'change': added, 'direction': 'added'}, {'year': year, 'change': 0 - removed, 'direction': 'removed'}])
    
    

In [ ]:
alt.Chart(pd.DataFrame(years)).mark_bar().encode(
    x='year:N',
    y='change:Q',
    color='direction:N'
)

In [ ]:
df_years

In [ ]:
df.loc[20912].to_list()

In [ ]:
df.loc[df['urlkey'] == 'au,gov,naa)/publications/fact_sheets/fs123.html'][200:]

In [ ]:
response = requests.get('https://web.archive.org/web/20200316010205id_/https://www.naa.gov.au/information-management/managing-information-and-records/types-information/social-media/index.aspx', headers={'User-Agent': ''})
response.status_code

In [ ]:
df.loc[df['mimetype'] == 'warc/revisit'].head(20)

In [ ]:
urls.loc[urls['year'] == 1998]['urlkey'].to_frame()

In [ ]:
df.loc[df['statuscode'] == '302']

In [ ]:
df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')]

In [ ]:
alt.Chart(df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')]).mark_bar().encode(
    x = 'month(date):T',
    y = 'count():Q'
)

In [ ]:
from urllib.parse import urlparse
import re

paths = []
for row in df.loc[(df['date'] >= '2007-01-01') & (df['date'] <= '2007-12-31')].itertuples():
    parsed = urlparse(row.original)
    paths += [w for w in re.split(r'\W+', parsed[2]) if w != '']

In [ ]:
from collections import Counter
c = Counter(paths)
c.most_common(50)

## Mimetypes and file extensions

In [ ]:
df['extension'] = df['original'].str.extract(r'\.(\w+)$')

In [ ]:
df['extension'].value_counts()[:25]

In [ ]:
df_formats = df.loc[df['extension'].isin(['html', 'css', 'jpg', 'gif', 'htm', 'pdf', 'png', 'mp3', 'txt', 'ppt', 'swf', 'doc'])]

In [ ]:
alt.Chart(df_formats).mark_bar().encode(
    x = 'year(date):T',
    y = 'count():Q',
    color = 'extension'
).properties(width=700)

In [ ]:
df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31') & (df['extension'] == 'pdf')]

In [ ]:
df.loc[df['original'].str.contains('/dsp/rft/model.pdf')]

In [ ]:
df.loc[(df['date'] >= '2000-01-01') & (df['date'] <= '2000-12-31') & (df['extension'] == 'gif')]

In [ ]:
df.loc[df['extension'] == 'pdf']['urlkey'].to_list()